In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
import pandas as pd
from datasets import load_dataset
import torch.nn.functional as F


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x111f27250>>
Traceback (most recent call last):
  File "/Users/nikhilsultania/Library/Caches/pypoetry/virtualenvs/nlp-group19-roMxBzuk-py3.11/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [10]:
DEVICE = torch.device("mps")

In [11]:
STUDENT_MODEL = "google-bert/bert-base-uncased"

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(STUDENT_MODEL, num_labels=3).to(DEVICE)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
model.config.id2label = {0: "background", 1: "result", 2: "method"}
model.config.label2id = {"background": 0, "result": 1, "method": 2}

In [9]:
train_dataset = pd.read_csv("../data/train_cleaned.csv")

In [14]:
train_dataset.head()

,Unnamed: 0,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,0,explicit,175.0,Introduction,168.0,"However, how frataxin interacts with the Fe-S ...",background,1.0,1872080baa7d30ec8fb87be9a65358cd3a7fb649,894be9b4ea46a5c422e81ef3c241072d4c73fdc0,True,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,11,NaN,NaN
1,1,explicit,36.0,Novel Quantitative Trait Loci for Seminal Root...,16.0,"In the study by Hickey et al. (2012), spikes w...",background,1.0,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b,b6642e19efb8db5623b3cc4eef1c5822a6151107,True,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,2,NaN,NaN
2,2,explicit,228.0,Introduction,225.0,"The drug also reduces catecholamine secretion,...",background,1.0,9cdf605beb1aa1078f235c4332b3024daa8b31dc,4e6a17fb8d7a3cada601d942e22eb5da6d01adbd,False,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,0,NaN,NaN
3,3,explicit,110.0,Discussion,46.0,By clustering with lowly aggressive close kin ...,background,1.0,d9f3207db0c79a3b154f3875c9760cc6b056904b,2cc6ff899bf17666ad35893524a4d61624555ed7,False,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,3,NaN,NaN
4,4,explicit,239.0,Discussion,234.0,Ophthalmic symptoms are rare manifestations of...,background,1.0,88b86556857f4374842d2af2e359576806239175,a5bb0ff1a026944d2a47a155462959af2b8505a8,False,88b86556857f4374842d2af2e359576806239175>a5bb0...,88b86556857f4374842d2af2e359576806239175>a5bb0...,2,NaN,NaN


In [15]:
labels = train_dataset["label"]
train_dataset = train_dataset.drop(columns=["Unnamed: 0", "label"])

In [16]:
input_strings = []
for i in range(len(train_dataset)):
    json_object = train_dataset.iloc[i].to_json()
    input_strings.append(str(json_object))
    

In [10]:
# TOBE PROCESSED LATER
# input_strings = [i for i in input_strings if len(i) <= 1024]

In [17]:
count = 0
for i in range(len(input_strings)):
    tokenized = tokenizer(input_strings[i], return_tensors="pt", padding="max_length", max_length=512, truncation=True).to(DEVICE) 
    with torch.no_grad():
        outputs = model(**tokenized)
    probs = F.softmax(outputs.logits, dim=1)
    predicted_class_id = torch.argmax(probs).item()
    label = model.config.id2label[predicted_class_id]
    if label == labels[i]:
        count += 1

KeyboardInterrupt: 

In [13]:
print(count)

2266


In [14]:
print(len(input_strings))

8194


In [19]:
dataset = load_dataset("csv", data_files="./results/Teachers/Ours/deepseek-openai/deepseek_openai_combined.csv")["train"].remove_columns("Unnamed: 0")
dataset = dataset.add_column("input", input_strings)
# dataset["train"] = dataset["train"].add_column("input", input_strings)

In [16]:
def process_data(row):
    input_text = row["input"]
    rationale = row["reasoning"]
    label = model.config.label2id[row["model_classification"]]

    input_enc = tokenizer(input_text, truncation=True, padding="max_length", max_length=512).to(DEVICE)
    rationale_enc = tokenizer(rationale, truncation=True, padding="max_length", max_length=512).to(DEVICE)

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": label,
        "rationale_ids": rationale_enc["input_ids"],
        "rationale_mask": rationale_enc["attention_mask"]
    }

In [17]:
processed_dataset = dataset.map(process_data)

Map: 100%|██████████| 8194/8194 [00:06<00:00, 1292.78 examples/s]


In [18]:
processed_dataset

Dataset({
    features: ['id', 'model_classification', 'reasoning', 'input', 'input_ids', 'attention_mask', 'labels', 'rationale_ids', 'rationale_mask'],
    num_rows: 8194
})

In [ ]:
from datasets import Dataset

processed_dataset.save_to_disk("processed_datset_example")

Saving the dataset (1/1 shards): 100%|██████████| 10/10 [00:00<00:00, 3234.10 examples/s]


In [19]:
training_args = TrainingArguments(
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=False,
)

In [20]:
class MultiTaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, **kwargs):
        labels = inputs.pop("labels")
        rationale_ids = inputs.pop("rationale_ids", None)
        
        outputs = model(**inputs)
        
        loss_fn = torch.nn.CrossEntropyLoss()
        label_loss = loss_fn(outputs.logits, labels)
        
        if rationale_ids is not None:
            rationale_outputs = model(input_ids=rationale_ids, attention_mask=inputs["attention_mask"])
            rationale_loss = loss_fn(rationale_outputs.logits, rationale_ids)
            loss = label_loss + 0.5 * rationale_loss  # Weighted loss
        else:
            loss = label_loss
        
        return (loss, outputs) if return_outputs else loss

trainer = MultiTaskTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=processed_dataset
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.704600,0.740657
2,0.529800,0.447692
3,0.405500,0.343345


TrainOutput(global_step=3075, training_loss=0.5804944064752843, metrics={'train_runtime': 3858.9428, 'train_samples_per_second': 6.37, 'train_steps_per_second': 0.797, 'total_flos': 6467854034589696.0, 'train_loss': 0.5804944064752843, 'epoch': 3.0})

In [22]:
count = 0
for i in range(len(input_strings)):
    tokenized = tokenizer(input_strings[i], return_tensors="pt", padding="max_length", max_length=512, truncation=True).to(DEVICE) 
    with torch.no_grad():
        outputs = model(**tokenized)
    probs = F.softmax(outputs.logits, dim=1)
    predicted_class_id = torch.argmax(probs).item()
    label = model.config.id2label[predicted_class_id]
    if label == labels[i]:
        count += 1

In [23]:
count

6179

In [153]:
string = input_strings[11]

In [154]:
string

'{"source":"explicit","citeEnd":101.0,"sectionName":"Introduction","citeStart":80.0,"string":"DOI: 10.7554\\/eLife.08828 1 of 27\\nmitochondrial division (Friedman et al., 2011; Korobova et al., 2013; Murley et al., 2013; Korobova et al., 2014).","label_confidence":0.7834,"citingPaperId":"929a7548464ce1d9d785abbe35da34842c0133e6","citedPaperId":"21096eb950bdff77308ad3924d69b681589bdc91","isKeyCitation":true,"id":"929a7548464ce1d9d785abbe35da34842c0133e6>21096eb950bdff77308ad3924d69b681589bdc91","unique_id":"929a7548464ce1d9d785abbe35da34842c0133e6>21096eb950bdff77308ad3924d69b681589bdc91_11","excerpt_index":11,"label2":null,"label2_confidence":null}'

In [155]:
tokenized = tokenizer(string, return_tensors="pt").to(DEVICE)

In [156]:
tokenized

{'input_ids': tensor([[  101,  1063,  1000,  3120,  1000,  1024,  1000, 13216,  1000,  1010,
          1000, 21893, 10497,  1000,  1024,  7886,  1012,  1014,  1010,  1000,
          2930, 18442,  1000,  1024,  1000,  4955,  1000,  1010,  1000, 17248,
          7559,  2102,  1000,  1024,  3770,  1012,  1014,  1010,  1000,  5164,
          1000,  1024,  1000,  9193,  1024,  2184,  1012,  4293, 27009,  1032,
          1013, 12005,  7959,  1012,  5511,  2620, 22407,  1015,  1997,  2676,
          1032, 22484,  3406, 24561, 13626,  4818,  2407,  1006, 18486,  3802,
          2632,  1012,  1010,  2249,  1025, 12849,  3217,  5092,  3567,  3802,
          2632,  1012,  1010,  2286,  1025, 14163, 12866,  3802,  2632,  1012,
          1010,  2286,  1025, 12849,  3217,  5092,  3567,  3802,  2632,  1012,
          1010,  2297,  1007,  1012,  1000,  1010,  1000,  3830,  1035,  7023,
          1000,  1024,  1014,  1012,  6275, 22022,  1010,  1000,  8951, 23298,
          3593,  1000,  1024,  1000,  

In [24]:
trainer_output = trainer.predict(processed_dataset)

In [25]:
trainer_output.predictions


array([[ 4.301876  , -2.359562  , -0.89601904],
       [-0.78595185, -1.7958096 ,  3.4897745 ],
       [ 4.282859  , -2.3421915 , -0.87154573],
       ...,
       [ 3.2073004 , -0.42851496, -1.4330306 ],
       [ 3.555149  , -0.9191982 , -1.2913921 ],
       [ 4.308708  , -2.3498607 , -0.8902831 ]],
      shape=(8194, 3), dtype=float32)

In [27]:
p = trainer_output.predictions.argmax(1)

In [28]:
p

array([0, 2, 0, ..., 0, 0, 0], shape=(8194,))

In [ ]:
model.config.id2label[p[0]]

'background'

In [32]:
new_count = 0
for i in range(len(labels)):
    if model.config.id2label[p[i]] == labels[i]:
        new_count += 1
print(new_count)

6179
